In [1]:
from google.cloud import bigquery
import re
import os
import pandas as pd

project_id = 'iter-data-storage-pv-uat'
client = bigquery.Client(project_id)

In [ ]:
path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Validaciones/Validacion Datalake/validacion de plataforma raw/query_igualdad/'

In [ ]:
path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Validaciones/Validacion Datalake/validacion de plataforma raw/query_igualdad/'

In [ ]:
path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Validaciones/Validacion Datalake/validacion de plataforma raw/query_null/'

In [ ]:
def read_table_bq():
  sql = """
          WITH TMP AS (
        SELECT
        COLUMN_NAME
        ,DATA_TYPE
        ,TABLE_NAME
        ,TABLE_CATALOG
        ,TABLE_SCHEMA
        ,CONCAT(TABLE_CATALOG,'.',TABLE_SCHEMA,'.',TABLE_NAME) AS ID_TABLE_UAT
        ,CONCAT('interseguro-data','.',TABLE_SCHEMA,'.',TABLE_NAME) AS ID_TABLE_PROD
        ,ROW_NUMBER()OVER(PARTITION BY TABLE_NAME) AS ORDEN
        ,CASE WHEN DATA_TYPE ='STRING' THEN CONCAT('TRIM(UPPER(PROD.',COLUMN_NAME,'))')
              ELSE CONCAT("PROD.",COLUMN_NAME) END AS VALOR_BUC
        ,CASE WHEN DATA_TYPE ='STRING' THEN CONCAT('TRIM(UPPER(UAT.',COLUMN_NAME,'))')
              ELSE CONCAT("UAT.",COLUMN_NAME) END AS VALOR_MIR
        FROM  iter-data-storage-pv-uat.acsele_data.INFORMATION_SCHEMA.COLUMNS
        --WHERE TABLE_NAME IN ('TD_POLIZA_ASEGURADO_DESGTARJETASIND_raw')
        --WHERE TABLE_NAME LIKE 'TD_%'
        --AND TABLE_NAME NOT LIKE '%_landing%'
        )
        SELECT
        ORDEN
        ,TABLE_NAME
        ,COLUMN_NAME
        ,ID_TABLE_UAT
        ,CASE WHEN ORDEN = 1 THEN CONCAT('SELECT SUM(CASE WHEN ',VALOR_BUC,'!=',VALOR_MIR,' THEN 1 ELSE 0 END) ','AS DIFF_',COLUMN_NAME)
            --  WHEN ORDEN = COUNT(*)OVER(PARTITION BY TABLE_NAME) THEN CONCAT (
            --  CONCAT(',SUM(CASE WHEN ',VALOR_BUC,'!=',VALOR_MIR,' THEN 1 ELSE 0 END) ','AS DIFF_',COLUMN_NAME)
            --  ,CONCAT (' FROM ',ID_TABLE_PROD,' PROD INNER JOIN ',
            --  ID_TABLE_UAT ,' UAT ON PROD.NUMERO_POLIZA = UAT.NUMERO_POLIZA AND PROD.NUMERO_OPERACION = UAT.NUMERO_OPERACION')
            --  )
              ELSE CONCAT(',SUM(CASE WHEN ',VALOR_BUC,'!=',VALOR_MIR,' THEN 1 ELSE 0 END) ','AS DIFF_',COLUMN_NAME) END AS QUERY
        FROM TMP
        ORDER BY TABLE_NAME, ORDEN ASC

  """
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
  return df

In [ ]:
def read_table_bq_null():
  sql = """
        WITH TMP AS (
        SELECT
         COLUMN_NAME
        ,DATA_TYPE
        ,TABLE_NAME
        ,TABLE_CATALOG
        ,TABLE_SCHEMA
        ,CONCAT(TABLE_CATALOG,'.',TABLE_SCHEMA,'.',TABLE_NAME) AS ID_TABLE_UAT
        ,CONCAT('interseguro-data','.',TABLE_SCHEMA,'.',TABLE_NAME) AS ID_TABLE_PROD
        ,ROW_NUMBER()OVER(PARTITION BY TABLE_NAME) AS ORDEN
        ,CASE WHEN DATA_TYPE ='STRING' THEN CONCAT('TRIM(UPPER(PROD.',COLUMN_NAME,'))')
              ELSE CONCAT("PROD.",COLUMN_NAME) END AS VALOR_BUC
        ,CASE WHEN DATA_TYPE ='STRING' THEN CONCAT('TRIM(UPPER(UAT.',COLUMN_NAME,'))')
              ELSE CONCAT("UAT.",COLUMN_NAME) END AS VALOR_MIR
        FROM  iter-data-storage-pv-uat.acsele_data.INFORMATION_SCHEMA.COLUMNS
        WHERE DATA_TYPE ='STRING' -- TABLE_NAME IN ('TD_OPERACION_VIDA_AGENTE_raw')
        )
        SELECT
        ORDEN
        ,TABLE_NAME
        ,COLUMN_NAME
        ,ID_TABLE_UAT
        ,IF(DATA_TYPE='STRING',(CASE WHEN ORDEN = 1 THEN CONCAT('SELECT SUM(CASE WHEN ',VALOR_BUC,' IS NULL AND ',VALOR_MIR,'="" THEN 1 ELSE 0 END) ','AS DIFF_',COLUMN_NAME)
                                          ELSE CONCAT(',SUM(CASE WHEN ',VALOR_BUC,' IS NULL AND ',VALOR_MIR,'="" THEN 1 ELSE 0 END) ','AS DIFF_',COLUMN_NAME)
                                          END ),NULL)  AS QUERY
        FROM TMP
        ORDER BY TABLE_NAME, ORDEN ASC
  """
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
  return df

In [ ]:
def read_table_bq_3():
  sql = """
        SELECT
        TABLE_NAME
        ,COLUMN_NAME
        ,AMBIENTE_PROD
        ,AMBIENTE_UAT
        from `iter-data-storage-pv.inputfiles.TABLES_ORACLE`
  """
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
  return df

In [ ]:
df=read_table_bq_null()

In [ ]:
df=read_table_bq()

In [ ]:
do=read_table_bq_3()

In [ ]:
grouped_data = df.groupby('TABLE_NAME')
for table_name, group in grouped_data:
    queries = '\n'.join(group['QUERY'])
    file_name = f'{table_name}.sql'
    with open(path+file_name, 'w') as file:
        file.write(queries)

In [ ]:
archivos_sql = [archivo for archivo in os.listdir(path) if archivo.endswith('.sql')]

In [ ]:
for archivo in archivos_sql:
    ruta_archivo = os.path.join(path, archivo)
    tabla_name = archivo.split('.')[0]
    with open(ruta_archivo, 'a') as file:
        condiciones_cruce = []
        for _, row in do[do['TABLE_NAME'] == tabla_name].iterrows():
            condiciones_cruce.append(f'UAT.{row["COLUMN_NAME"]} = PROD.{row["COLUMN_NAME"]}')
        if condiciones_cruce:
            file.write('\n'+f'FROM {row["AMBIENTE_UAT"]} UAT '+'\n')
            file.write(f'INNER JOIN {row["AMBIENTE_PROD"]} PROD ')
            file.write('\nON ' + ' \nAND '.join(condiciones_cruce))